In [31]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Set seed for reproducibility
random.seed(42)

# Define parameters
num_rows = 1000
num_customers = 100
countries = ["Kenya", "Uganda", "Tanzania", "Rwanda", "Ethiopia", "South Africa", "Nigeria", "Ghana", "Zambia", "Botswana"]
products = [
    ("P001", "Laptop"), ("P002", "Smartphone"), ("P003", "Desk Chair"),
    ("P004", "Notebook"), ("P005", "Pen"), ("P006", "Headphones"),
    ("P007", "Monitor"), ("P008", "Backpack"), ("P009", "Mouse"), ("P010", "Keyboard")
]

# Generate synthetic data
data = []
start_date = datetime(2023, 8, 12)
end_date = datetime(2025, 8, 12)

for _ in range(num_rows):
    invoice_no = f"INV{random.randint(100000, 999999)}"
    stock_code, description = random.choice(products)
    quantity = random.randint(1, 50)
    unit_price = round(random.uniform(1.0, 100.0), 2)
    invoice_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    customer_id = random.randint(1, num_customers)
    country = random.choice(countries)

    data.append([
        invoice_no, stock_code, description, quantity,
        invoice_date, unit_price, customer_id, country
    ])

# Create DataFrame
df = pd.DataFrame(data, columns=[
    "InvoiceNo", "StockCode", "Description", "Quantity",
    "InvoiceDate", "UnitPrice", "CustomerID", "Country"
])

# Convert InvoiceDate to datetime (already done, but for safety)
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Preview first few rows
print("Synthetic data generated:")
print(df.head())


Synthetic data generated:
   InvoiceNo StockCode Description  Quantity InvoiceDate  UnitPrice  \
0  INV770487      P002  Smartphone         2  2024-04-18      74.41   
1  INV872246      P002  Smartphone        44  2025-02-20      74.33   
2  INV542417      P001      Laptop         2  2024-04-06      10.28   
3  INV127824      P009       Mouse        13  2025-07-30      71.89   
4  INV331148      P008    Backpack        38  2023-08-18      28.54   

   CustomerID   Country  
0          29  Tanzania  
1          12  Botswana  
2          65  Botswana  
3          70   Nigeria  
4          98  Tanzania  


In [32]:
# Add TotalSales column
df["TotalSales"] = df["Quantity"] * df["UnitPrice"]

# Remove outliers
df_clean = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]

# Filter for sales in the last year
cutoff_date = pd.Timestamp("2024-08-12")
df_recent = df_clean[df_clean["InvoiceDate"] > cutoff_date]

# Create Customer Summary
customer_summary = df_recent.groupby("CustomerID").agg({
    "TotalSales": "sum",
    "Country": "first"  # Assuming each customer is tied to one country
}).reset_index()

# Preview transformed data
print("Transformed Sales Data (Recent):")
print(df_recent.head())

print("\nCustomer Summary:")
print(customer_summary.head())


Transformed Sales Data (Recent):
    InvoiceNo StockCode Description  Quantity InvoiceDate  UnitPrice  \
1   INV872246      P002  Smartphone        44  2025-02-20      74.33   
3   INV127824      P009       Mouse        13  2025-07-30      71.89   
8   INV678856      P005         Pen        41  2024-08-16      62.23   
10  INV205907      P007     Monitor        18  2024-08-19      45.89   
11  INV472528      P004    Notebook        43  2025-07-11      27.43   

    CustomerID       Country  TotalSales  
1           12      Botswana     3270.52  
3           70       Nigeria      934.57  
8           74        Rwanda     2551.43  
10          21  South Africa      826.02  
11          83        Uganda     1179.49  

Customer Summary:
   CustomerID  TotalSales       Country
0           1     7527.79  South Africa
1           2    12012.99        Zambia
2           3     9366.79        Uganda
3           4     2367.62        Uganda
4           5     6511.92        Zambia


In [33]:
# Load data into SQLite database
import sqlite3

# Connect to database
conn = sqlite3.connect("retail_dw.db")
cursor = conn.cursor()

# Create tables using original schema
cursor.execute("""
CREATE TABLE IF NOT EXISTS CustomerDim (
    CustomerID INTEGER PRIMARY KEY,
    Name TEXT,
    Location TEXT,
    Age INTEGER,
    Gender TEXT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS ProductDim (
    ProductID INTEGER PRIMARY KEY,
    ProductName TEXT,
    Category TEXT,
    UnitPrice REAL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS TimeDim (
    TimeID INTEGER PRIMARY KEY,
    Date TEXT,
    Month INTEGER,
    Quarter INTEGER,
    Year INTEGER
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS SalesFact (
    SaleID INTEGER PRIMARY KEY AUTOINCREMENT,
    CustomerID INTEGER,
    ProductID INTEGER,
    TimeID INTEGER,
    Quantity INTEGER,
    SalesAmount REAL,
    FOREIGN KEY (CustomerID) REFERENCES CustomerDim(CustomerID),
    FOREIGN KEY (ProductID) REFERENCES ProductDim(ProductID),
    FOREIGN KEY (TimeID) REFERENCES TimeDim(TimeID)
);
""")

conn.commit()


    


In [36]:
# Transform CustomerDim Data
import pandas as pd
import random

# Use customer_summary from your earlier transformation
# Assume df_recent and customer_summary already exist

# Generate synthetic attributes for each customer
def generate_customer_dim(customer_summary):
    random.seed(42)
    names_pool = ['Alice', 'Bob', 'Charlie', 'Diana', 'Ethan', 'Fiona', 'George', 'Hannah', 'Ian', 'Julia']
    locations = customer_summary['Country'].unique().tolist()
    genders = ['Male', 'Female']

    customer_dim_data = []
    for i, row in customer_summary.iterrows():
        name = random.choice(names_pool) + str(row['CustomerID'])  # Ensure uniqueness
        location = row['Country']
        age = random.randint(18, 65)
        gender = random.choice(genders)

        customer_dim_data.append((
            int(row['CustomerID']),
            name,
            location,
            age,
            gender
        ))

    return pd.DataFrame(customer_dim_data, columns=['CustomerID', 'Name', 'Location', 'Age', 'Gender'])

# Generate the dimension
customer_dim_df = generate_customer_dim(customer_summary)

# Preview
print("CustomerDim DataFrame:")
print(customer_dim_df.head())


CustomerDim DataFrame:
   CustomerID    Name      Location  Age  Gender
0           1    Bob1  South Africa   19  Female
1           2  Diana2        Zambia   32    Male
2           3    Bob3        Uganda   61    Male
3           4  Julia4        Uganda   45    Male
4           5  Alice5        Zambia   23    Male


In [ ]:
import sqlite3

# Connect to database
conn = sqlite3.connect("retail_dw.db")
cursor = conn.cursor()

# Clear existing data
cursor.execute("DELETE FROM CustomerDim")
conn.commit()

# Insert cleaned customer data
for _, row in customer_dim_df.iterrows():
    cursor.execute("""
        INSERT INTO CustomerDim (CustomerID, Name, Location, Age, Gender)
        VALUES (?, ?, ?, ?, ?)
    """, tuple(row))

conn.commit()

# Preview inserted data
print("Preview of CustomerDim:")
preview_df = pd.read_sql_query("SELECT * FROM CustomerDim LIMIT 5", conn)
print(preview_df)

conn.close()


Preview of CustomerDim:
   CustomerID    Name      Location  Age  Gender
0           1    Bob1  South Africa   19  Female
1           2  Diana2        Zambia   32    Male
2           3    Bob3        Uganda   61    Male
3           4  Julia4        Uganda   45    Male
4           5  Alice5        Zambia   23    Male


In [37]:
# Transform and clean ProductDim Data
import pandas as pd
import random

# Define categories
category_map = {
    "Laptop": "Electronics",
    "Smartphone": "Electronics",
    "Desk Chair": "Furniture",
    "Notebook": "Stationery",
    "Pen": "Stationery",
    "Headphones": "Electronics",
    "Monitor": "Electronics",
    "Backpack": "Accessories",
    "Mouse": "Electronics",
    "Keyboard": "Electronics"
}

# Generate product dimension data
product_dim_data = []
for i, (code, name) in enumerate(products, start=1):
    category = category_map.get(name, "Misc")
    unit_price = round(random.uniform(10.0, 1000.0), 2)
    product_dim_data.append((i, name, category, unit_price))

product_dim_df = pd.DataFrame(product_dim_data, columns=['ProductID', 'ProductName', 'Category', 'UnitPrice'])

# Preview
print("ProductDim DataFrame:")
print(product_dim_df.head())


ProductDim DataFrame:
   ProductID ProductName     Category  UnitPrice
0          1      Laptop  Electronics     935.36
1          2  Smartphone  Electronics     212.22
2          3  Desk Chair    Furniture     719.03
3          4    Notebook   Stationery     246.30
4          5         Pen   Stationery     401.83


In [39]:
# Transform and clean TimeDim Data
# Extract unique dates from recent sales
unique_dates = df_recent["InvoiceDate"].drop_duplicates().sort_values().reset_index(drop=True)

# Build TimeDim records
time_dim_data = []
for i, date in enumerate(unique_dates, start=1):
    time_dim_data.append((
        i,
        date.strftime("%Y-%m-%d"),
        date.month,
        (date.month - 1) // 3 + 1,
        date.year
    ))

time_dim_df = pd.DataFrame(time_dim_data, columns=["TimeID", "Date", "Month", "Quarter", "Year"])

# Preview
print("TimeDim DataFrame:")
print(time_dim_df.head())


TimeDim DataFrame:
   TimeID        Date  Month  Quarter  Year
0       1  2024-08-13      8        3  2024
1       2  2024-08-14      8        3  2024
2       3  2024-08-15      8        3  2024
3       4  2024-08-16      8        3  2024
4       5  2024-08-17      8        3  2024


In [45]:
# Reconnect to database
conn = sqlite3.connect("retail_dw.db")
cursor = conn.cursor()

# Load lookup tables
product_lookup = pd.read_sql_query("SELECT ProductID, ProductName FROM ProductDim", conn)
time_lookup = pd.read_sql_query("SELECT TimeID, Date FROM TimeDim", conn)

# Prepare df_recent for joins
df_recent = df[df["InvoiceDate"] >= "2022-01-01"]  # Filter recent transactions

# ✅ Make a clean copy to avoid SettingWithCopyWarning
df_fact = df_recent.copy()

# Format date for joining with TimeDim
df_fact["Date"] = df_fact["InvoiceDate"].dt.strftime("%Y-%m-%d")

# Merge ProductID using Description as ProductName
df_fact = df_fact.merge(product_lookup, left_on="Description", right_on="ProductName", how="left")

# Merge TimeID
df_fact = df_fact.merge(time_lookup, on="Date", how="left")

# Calculate SalesAmount
df_fact["SalesAmount"] = df_fact["Quantity"] * df_fact["UnitPrice"]

# Select final columns
sales_fact_df = df_fact[["CustomerID", "ProductID", "TimeID", "Quantity", "SalesAmount"]].copy()

# Insert into SalesFact
cursor.execute("DELETE FROM SalesFact")
conn.commit()

for _, row in sales_fact_df.iterrows():
    cursor.execute("""
        INSERT INTO SalesFact (CustomerID, ProductID, TimeID, Quantity, SalesAmount)
        VALUES (?, ?, ?, ?, ?)
    """, tuple(row))

conn.commit()
print("SalesFact table populated with", len(sales_fact_df), "records.")
conn.close()


SalesFact table populated with 1000 records.


In [44]:
print(df_recent.head())
print(customer_dim.head())


   InvoiceNo StockCode Description  Quantity InvoiceDate  UnitPrice  \
0  INV770487      P002  Smartphone         2  2024-04-18      74.41   
1  INV872246      P002  Smartphone        44  2025-02-20      74.33   
2  INV542417      P001      Laptop         2  2024-04-06      10.28   
3  INV127824      P009       Mouse        13  2025-07-30      71.89   
4  INV331148      P008    Backpack        38  2023-08-18      28.54   

   CustomerID   Country  TotalSales  
0          29  Tanzania      148.82  
1          12  Botswana     3270.52  
2          65  Botswana       20.56  
3          70   Nigeria      934.57  
4          98  Tanzania     1084.52  
   CustomerID    Name      Location  Age  Gender
0           1    Bob1  South Africa   19  Female
1           2  Diana2        Zambia   32    Male
2           3    Bob3        Uganda   61    Male
3           4  Julia4        Uganda   45    Male
4           5  Alice5        Zambia   23    Male
